# Ошибки случаются или работа с исключениями.



## 0. Мотивация



Пока мы пишем и исполняем код, мы так или иначе будем натыкаться на ошибки. Это нормально. Другой вопрос, хотят ли другие люди видеть ошибки во время исполнения нашего кода и как они хотят их видеть?

Давайте представим, что мы пишем любой код для не-программистов. Внезапно что-то происходит и наша программа не может дальше работать верно. *Отдельный вопрос - по чьей вине: пользователь мог ввести неверные данные или мы могли допустить ошибку, которая не позволяет работать программе при введенных данных.* Хочет ли пользователь нашей программы видеть всякие ошибки типа `ValueError` и разбираться, что же произошло? Конечно же, нет. Пользователю будет гораздо понятнее, если вместо стандартной ошибки написать "Введено некорректное значение" или что-то похожее. Поэтому многие типичные ошибки мы хотим "отлавливать" самостоятельно, чтобы с ними что-то сделать.

Кроме того, когда возникают ошибки, важно их "записывать" (корректный термин здесь - логировать), чтобы с ними мог разобраться потом кто-то компетентный. Это еще один повод специальным образом обрабатывать ошибки.

### Пропустим вопрос "кто виноват?" и спросим "что делать?"

Варианта у нас два: либо предотвращать возможные ошибки, либо устранять проблемы по мере поступления.

## 1. LBYL

Один из возможных способов обрабатывать ошибки - проверять данные на корректность *перед* тем, как что-то с ними делать. 

Такой подход называется `LBYL` или `Look Before You Leap` (предпочитаю переводить как "смотри, куда прыгаешь").

То есть прежде, чем сделать что-то, мы проверяем, всё ли в норме. Если не задать все возможные (необходимые) вопросы, то случится что-нибудь плохое.

Такой подход не идеален по нескольким причинам (цитируя Alex Martelli "Python in a Nutshell"):

1) Проверки могут ухудшить читабельность и понятность кода. (А тем более стандартных ситуаций, когда всё и так в порядке)
2) Вычисления, необходимые для проверок, могут дублировать большую часть работы самой операции, перед которой мы делаем проверки.
3) Программист может легко ошибиться и упустить некоторые необходимые проверки
4) Ситуация может измениться между моментом проверки и моментом выполнения операции (примеры - `os.path.exists` или `Queue.full`)

Пример (очевидно, надуманный, но показывающий общий антипаттерн):

In [16]:
def print_object(some_object):
    # Проверяем, что объект можно распечатать...
    if isinstance(some_object, str):
        print(some_object)
    elif isinstance(some_object, dict):
        print(some_object)
    elif isinstance(some_object, list):
        print(some_object)
    # и еще 97 elif'ов...
    else:
        print("Нельзя распечатать")


А это всего лишь простая функция, проверяющая, можно ли распечатать объект. Попытка проверить все условия, в которых возникает ошибка, как будто бы заведомо обречена на провал (так еще и код становится уродливым и нечитабельным). 

Поэтому мы переходим ко второму подходу, свойственному в основном `Python` (потому что во многих других языках (в основном компилируемых) написание исключений сильно влияет на производительность, а потому считается скорее плохой практикой).

## 2. EAFP

`Easier to Ask for Forgiveness than Permission` - "проще просить прощения, чем разрешения". 

Идея проста - сначала пробуем что-то сделать, а если не получается - начинаем извиняться и пытаемся разобраться.

Прошлый код может быть переписан так:

In [17]:
def print_object(some_object):
    # Проверяем, что объект можно распечатать
    try:
        printable = str(some_object)
        print(printable)
    except TypeError:
        print("Нельзя распечатать")


Более того, предусмотрена конструкция с `else`, которая позволяет написать еще более качественный код:

In [18]:
def print_object(some_object):
    # Проверяем, что объект можно распечатать
    try:
        printable = str(some_object)
    except TypeError:
        print("unprintable object")
    else:
        print(printable)


*Заметка про улучшение кода. Наша проверка подразумевала именно возможность конвертации объекта в строку, но что если проблема бы возникла именно в помент вызова `print`? (ну мало ли)*

Теперь, `print` будет вызвана только если ошибки не возникло (и если проблема именно в вызове `print`, то ошибка выскочит как обычно).

## 3. А что с производительностью?

Прежде, чем переходить к деталям применения исключений, убедимся в том, что их использование допустимо в чувствительном к производительности коде.


Чтобы измерить влияние на производительность, нам необходимо измерить две (с половиной) вещи:
1) Накладные расходы на `try` блок, который никогда не выкидывает исключение
2) Накладные расходы на обработку исключения против эквивалентного кода без использования исключений

    а) Когда исключения случаются часто
    
    б) Когда исключения случаются редко

Для тестов будем использовать стандартную библиотеку `timeit`. 

Официальная документация [timeit](https://docs.python.org/3/library/timeit.html) подсказывает, что минимальное время, которое должен исполняться код для репрезентативных результатов, равно 0.2 секунды, поэтому повторяем действия необходимое количество раз.

Еще один момент: в `Timer.repeat` мы используем минимальное время выполнения из всех прогонов. Почему так, хотя казалось бы надо посчитать среднее время и стандартное отклонение?  Потому что часто время, которое больше минимального, связано не с различной скоростью выполнения кода, а с другими процессами в компьютере, которые могут влиять на время выполнения. Так что берем минимум.

Из документации:

> Note: it's tempting to calculate mean and standard deviation from the result vector and report these. However, this is not very useful. In a typical case, the lowest value gives a lower bound for how fast your machine can run the given code snippet; higher values in the result vector are typically not caused by variability in Python's speed, but by other processes interfering with your timing accuracy. So the min() of the result is probably the only number you should be interested in. After that, you should look at the entire vector and apply common sense rather than statistics.

### 3.1 Тест 1
Проверим накладные расходы на блок `try`, который никогда не использует конструкцию `except`.

Код ниже просто увеличивает счетчик на 1 двумя способами: без `try-except` и с ним. 

In [3]:
SETUP = "counter = 0"

LOOP_IF = """
counter += 1
"""

LOOP_EXCEPT = """
try:
    counter += 1
except:
    pass
"""

import timeit
number_of_iterations = 10**8

if_time = timeit.Timer(LOOP_IF, setup=SETUP)
except_time = timeit.Timer(LOOP_EXCEPT, setup=SETUP)

min_if_time = min(if_time.repeat(number=number_of_iterations))
min_except_time = min(except_time.repeat(number=number_of_iterations))

print(f"""Используя if:
      Среднее время выполнения: {min_if_time / number_of_iterations}
      """)
print(f"""Используя exception: 
      Среднее время выполнения: {min_except_time / number_of_iterations}
      """)


Используя if:
      Среднее время выполнения: 1.867625600018073e-08
      
Используя exception: 
      Среднее время выполнения: 2.1865148000069895e-08
      


Разница минимальна (порядка $3 \cdot 10^{-9}$ секунд на моей машине), но `try` стабильно медленнее.

### 3.2 Тест 2

Теперь сравним проверку с помощью `if` и конструкцию `try-except`, которая возникает с заданной вероятностью. 

Возьмем файл `words.txt`, который в `linux` обычно лежит в `/usr/share/dict/words` (он используется для спеллчекинга), из него возьмем определенный процент слов (это отношение хранится в переменной `percentage`) и создадим словарь. Вхождения в этот словарь и будем проверять: в первом случае - условием, а во втором - с помощью `try`. 

Получается, в зависимости от `percentage`, мы будем попадать в конструкцию `except` с заданной вероятностью ($1 - $`percentage`). 

In [5]:
import timeit

SETUP = """
import random
with open('./textfiles/words.txt', 'r') as fp:
    words = [word.strip() for word in fp.readlines()]
percentage = int(len(words) * 0.1)
my_dict = dict([(w, w) for w in random.sample(words, percentage)])
counter = 0
"""

LOOP_IF = """
word = random.choice(words)
if word in my_dict:
    counter += len(my_dict[word])
"""

LOOP_EXCEPT = """
word = random.choice(words)
try:
    counter += len(my_dict[word])
except KeyError:
    pass
"""

num_of_iters_test_2 = 10**7

if_time = timeit.Timer(LOOP_IF, setup=SETUP)
except_time = timeit.Timer(LOOP_EXCEPT, setup=SETUP)
min_if_time_test_2 = min(if_time.repeat(number=num_of_iters_test_2))
min_except_time_test_2 = min(except_time.repeat(number=num_of_iters_test_2))

print(f"""Используя if: 
      Среднее время выполнения: {min_if_time_test_2 / num_of_iters_test_2}
      """)
print(f"""Используя exception: 
      Среднее время выполнения: {min_except_time_test_2 / num_of_iters_test_2}
      """)

Используя if: 
      Среднее время выполнения: 3.203243200026918e-07
      
Используя exception: 
      Среднее время выполнения: 7.090981699991972e-07
      


Разница получилась двукратная. *Какой ужас, исключения вдважды медленнее!* 

Секундочку. Это $10^7$ повторов в цикле с шансом 90% вызвать исключение. Да, они немного медленнее, но будет ли хоть одна реальная ситуация, вызывающая исключения с таким шансом? Нет. Если наш код будет в 90% падать с исключением, то разумнее его переписать так, чтобы исключения не возникали настолько часто. 

А если шанс уменьшить до, например, 20%, то мы снова получим исчезающе малую разницу (на моей машине она равна $7 \cdot 10^{-8}$). 

Такая разница не важная никому (а если кому-то и важна, то этот человек зря читает данный текст, потому что подходящее чтиво для него - это "Язык программирования С" Ритчи и Кернигана).

#### Итог тестов - исключения замедляют выполнение кода на пренебрежительно маленькую величину.

## 4. Как применять `try-except`?


### 4.1 Пишите конкретные ошибки:

Конструкция с `except:` допустима, но это **не лучшая практика**.

#### Сравним:

Первый код перенаправляет ВСЕ ошибки в except и без отдельных команд не разберёшь, в чем дело. Такое бывает нужно, но это плохая практика.

In [21]:
try:
    some_digit = int(input())
except:
    print('Без понятия, что пошло не так')

Без понятия, что пошло не так


Второй код отлавливает конкретную ошибку и отдельно с ней работает. Замечу, что если хочется всё же выкинуть ошибку, то можно воспользоваться `raise`.

In [22]:
try:
    some_digit = int(input())
except ValueError:
    print('Невозможно конвертировать введенную строку в int')
    raise

Невозможно конвертировать введенную строку в int


ValueError: invalid literal for int() with base 10: 'g'

### 4.2 В блоке `try` пишите только то, что может выкинуть ошибку. 

Для остального используйте `else`.

Для того, что сделать нужно вне зависимости от того, возникла ли ошибка, используйте блок `finally`.

`except`'ов может быть несколько.

In [12]:
a = input("Введите число: ")
b = input("Введите второе число: ")
try:
    result = int(a) / int(b)
except ValueError:
    print("Невозможно конвертировать в int. Введите целые числа.")
except ZeroDivisionError:
    print("На ноль делить нельзя")
else:
    print(result)
finally:
    print("ВСЕ ЗАВЕРШИЛОСЬ")


На ноль делить нельзя
ВСЕ ЗАВЕРШИЛОСЬ


## 5. Ссылки

Официальная документация: https://docs.python.org/3/tutorial/errors.html

Хорошая статья, из которой взяты тесты производительности:
https://jeffknupp.com/blog/2013/02/06/write-cleaner-python-use-exceptions/

Простенький материал с примерами:
https://pythonchik.ru/osnovy/python-try-except

Чуть более подробный материал: https://academy.yandex.ru/handbook/python/article/model-isklyuchenij-python-try-except-else-finally-moduli

Хорошее видео кратко про принцип EAFP: https://www.youtube.com/watch?v=f-GFxTlqD2Q

Более подробное видео об исключениях (подробнее, чем мое изложение): https://www.youtube.com/watch?v=89wpfOAgrCk